## w205 Project 2
### Jacob Sycoff
#### Due: 10/29/2020

## Starter Code:

In [87]:
import json
import pprint

In [88]:
p = pprint.PrettyPrinter(indent=1)

In [89]:
f = open("assessment-attempts-20180128-121051-nested.json","r")


In [90]:
s = f.read()

In [91]:
assessments= json.loads(s)

In [92]:
f.close()

In [93]:
len(assessments)

3280

In [94]:
# this will pretty print the json in alphabetic order which may or may not match the file order
p.pprint(assessments[2])

{'base_exam_id': '4beeac16-bb83-4d58-83e4-26cdc38f0481',
 'certification': 'false',
 'exam_name': 'The Principles of Microservices',
 'keen_created_at': '1516738973.653394',
 'keen_id': '5a67999d3ed3e300016ef0f1',
 'keen_timestamp': '1516738973.653394',
 'max_attempts': '1.0',
 'sequences': {'attempt': 1,
               'counts': {'all_correct': False,
                          'correct': 3,
                          'incomplete': 0,
                          'incorrect': 1,
                          'submitted': 4,
                          'total': 4,
                          'unanswered': 0},
               'id': 'b370a3aa-bf9e-4c10-848a-8ecacbd1d93e',
               'questions': [{'id': 'b9ff2e88-cf9d-4bd4-bcea-c2673779425c',
                              'options': [{'at': '2018-01-23T20:22:39.089Z',
                                           'checked': True,
                                           'id': 'c315723f-4753-4f8f-932e-d9d7085fed28',
                                 

In [95]:
def recursive_walk_json_object(j, level):
    """recursively walk through a json object to explore the structure
       dictionaries will be put in alphabetic order to match the pretty print above"""
    
    level += 1
    
    if type(j) is dict:
        dict_2_list = list(j.keys())
        dict_2_list.sort()
        for k in dict_2_list:
            print("   " * level + "L" + str(level), k)
            recursive_walk_json_object(j[k], level)
    
    elif type(j) is list:
        for (i, l) in enumerate(j):
            print("  " * level + "  [" + str(i) + "]")
            recursive_walk_json_object(l, level)
            
    else:
        print("   " * level + " value:", j)

In [96]:
recursive_walk_json_object(assessments[0], -1)

L0 base_exam_id
    value: 37f0a30a-7464-11e6-aa92-a8667f27e5dc
L0 certification
    value: false
L0 exam_name
    value: Normal Forms and All That Jazz Master Class
L0 keen_created_at
    value: 1516717442.735266
L0 keen_id
    value: 5a6745820eb8ab00016be1f1
L0 keen_timestamp
    value: 1516717442.735266
L0 max_attempts
    value: 1.0
L0 sequences
   L1 attempt
       value: 1
   L1 counts
      L2 all_correct
          value: False
      L2 correct
          value: 2
      L2 incomplete
          value: 1
      L2 incorrect
          value: 1
      L2 submitted
          value: 4
      L2 total
          value: 4
      L2 unanswered
          value: 0
   L1 id
       value: 5b28a462-7a3b-42e0-b508-09f3906d1703
   L1 questions
      [0]
         L3 id
             value: 7a2ed6d3-f492-49b3-b8aa-d080a8aad986
         L3 options
          [0]
               L5 at
                   value: 2018-01-23T14:23:24.670Z
               L5 checked
                   value: True
               L

In [97]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load()

In [98]:
raw_assessments.cache()

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [99]:
assessments = raw_assessments.select(raw_assessments.value.cast('string'))

In [100]:
#i added this cell
print(spark.catalog.listTables())

[Table(name='assessments', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='ct', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='questions', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='sequences', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='ut', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [101]:
from pyspark.sql import Row

In [102]:
extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()

In [103]:
extracted_assessments.registerTempTable('assessments')

In [104]:
#i added this cell
print(spark.catalog.listTables())

[Table(name='assessments', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='ct', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='questions', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='sequences', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='ut', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [105]:
spark.sql("select keen_id from assessments limit 10").show()

+--------------------+
|             keen_id|
+--------------------+
|5a6745820eb8ab000...|
|5a674541ab6b0a000...|
|5a67999d3ed3e3000...|
|5a6799694fc7c7000...|
|5a6791e824fccd000...|
|5a67a0b6852c2a000...|
|5a67b627cc80e6000...|
|5a67ac8cb0a5f4000...|
|5a67a9ba060087000...|
|5a67ac54411aed000...|
+--------------------+



In [106]:
spark.sql("select keen_timestamp, sequences.questions[0].user_incomplete from assessments limit 10").show()

+------------------+-------------------------------------------------------+
|    keen_timestamp|sequences[questions] AS `questions`[0][user_incomplete]|
+------------------+-------------------------------------------------------+
| 1516717442.735266|                                                   true|
| 1516717377.639827|                                                  false|
| 1516738973.653394|                                                  false|
|1516738921.1137421|                                                  false|
| 1516737000.212122|                                                  false|
| 1516740790.309757|                                                  false|
|1516746279.3801291|                                                  false|
| 1516743820.305464|                                                  false|
|  1516743098.56811|                                                  false|
| 1516743764.813107|                                                  false|

In [107]:
def my_lambda_sequences_id(x):
    raw_dict = json.loads(x.value)
    my_dict = {"keen_id" : raw_dict["keen_id"], "sequences_id" : raw_dict["sequences"]["id"]}
    return Row(**my_dict)

In [108]:
my_sequences = assessments.rdd.map(my_lambda_sequences_id).toDF()

In [109]:
my_sequences.registerTempTable('sequences')

In [110]:
spark.sql("select sequences_id from sequences limit 10").show()

+--------------------+
|        sequences_id|
+--------------------+
|5b28a462-7a3b-42e...|
|5b28a462-7a3b-42e...|
|b370a3aa-bf9e-4c1...|
|b370a3aa-bf9e-4c1...|
|04a192c1-4f5c-4ac...|
|e7110aed-0d08-4cb...|
|5251db24-2a6e-424...|
|066b5326-e547-4da...|
|8ac691f8-8c1a-403...|
|066b5326-e547-4da...|
+--------------------+



In [111]:
spark.sql("select a.keen_id, a.keen_timestamp, s.sequences_id from assessments a join sequences s on a.keen_id = s.keen_id limit 10").show(truncate = False)

+------------------------+------------------+------------------------------------+
|keen_id                 |keen_timestamp    |sequences_id                        |
+------------------------+------------------+------------------------------------+
|5a17a67efa125700015e3164|1511499390.3836269|8ac691f8-8c1a-4033-b2e2-44e165775992|
|5a26ee9cbf5ce1000131c8e9|1512500892.4166169|9bd87823-4508-4e09-a376-8b81bd40f207|
|5a29dcac74b6620001da2dcf|1512692908.8423469|e7110aed-0d08-4cb3-9eca-3eb7caad0256|
|5a2fdab0eabeda0001ad6d97|1513085616.2275269|cd800e92-afc3-4473-a6aa-6cad1aac6be1|
|5a30105020e9d400013db76c|1513099344.8624721|8ac691f8-8c1a-4033-b2e2-44e165775992|
|5a3a6fc3f0a10000019bf958|1513779139.354213 |e7110aed-0d08-4cb3-9eca-3eb7caad0256|
|5a4e17fe08a8920001743e5f|1515067390.1336551|9abd5b51-6bd8-11e6-9f64-a8667f27e5dc|
|5a4f3c69cc6444000130dd2d|1515142249.858722 |083844c5-772f-48df-9a41-8cffd27dcae0|
|5a51b21bd0480b000126ab02|1515303451.773272 |e7110aed-0d08-4cb3-9eca-3eb7caad0256|
|5a5

In [112]:
def my_lambda_questions(x):
    raw_dict = json.loads(x.value)
    my_list = []
    my_count = 0
    for l in raw_dict["sequences"]["questions"]:
        my_count += 1
        my_dict = {"keen_id" : raw_dict["keen_id"], "my_count" : my_count, "id" : l["id"]}
        my_list.append(Row(**my_dict))
    return my_list

In [113]:
my_questions = assessments.rdd.flatMap(my_lambda_questions).toDF()

In [114]:
my_questions.registerTempTable('questions')

In [115]:
spark.sql("select id, my_count from questions limit 10").show(truncate=False)

+------------------------------------+--------+
|id                                  |my_count|
+------------------------------------+--------+
|7a2ed6d3-f492-49b3-b8aa-d080a8aad986|1       |
|bbed4358-999d-4462-9596-bad5173a6ecb|2       |
|e6ad8644-96b1-4617-b37b-a263dded202c|3       |
|95194331-ac43-454e-83de-ea8913067055|4       |
|95194331-ac43-454e-83de-ea8913067055|1       |
|bbed4358-999d-4462-9596-bad5173a6ecb|2       |
|e6ad8644-96b1-4617-b37b-a263dded202c|3       |
|7a2ed6d3-f492-49b3-b8aa-d080a8aad986|4       |
|b9ff2e88-cf9d-4bd4-bcea-c2673779425c|1       |
|bec23e7b-4870-49f7-92c1-1e0a016ef7c9|2       |
+------------------------------------+--------+



In [116]:
spark.sql("select q.keen_id, a.keen_timestamp, q.id from assessments a join questions q on a.keen_id = q.keen_id limit 10").show(truncate=False)

+------------------------+------------------+------------------------------------+
|keen_id                 |keen_timestamp    |id                                  |
+------------------------+------------------+------------------------------------+
|5a17a67efa125700015e3164|1511499390.3836269|803fc93f-7eb2-4121-af8c-ff809e850f10|
|5a17a67efa125700015e3164|1511499390.3836269|f3cb88cc-5b79-41bb-96d3-7ec7d698b964|
|5a17a67efa125700015e3164|1511499390.3836269|32fe7d8d-6d89-4db4-a17a-a368c5ea3ca0|
|5a17a67efa125700015e3164|1511499390.3836269|5c34cf19-8cfd-4f56-91c2-0a109dc990b9|
|5a26ee9cbf5ce1000131c8e9|1512500892.4166169|0603e6f4-c3f9-4c23-8cb3-eb43c68d9113|
|5a26ee9cbf5ce1000131c8e9|1512500892.4166169|26a06b88-2758-45bc-93c5-5e48deebb2ba|
|5a26ee9cbf5ce1000131c8e9|1512500892.4166169|25b6effe-79b0-4c47-83d0-2b5cfd655dd3|
|5a26ee9cbf5ce1000131c8e9|1512500892.4166169|6de03a9b-2a78-46b6-9c47-284477b57f28|
|5a26ee9cbf5ce1000131c8e9|1512500892.4166169|aaf39991-fa83-470f-b0f0-c548ef3a2eba|
|5a2

In [117]:
def my_lambda_correct_total(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:
        
        if "counts" in raw_dict["sequences"]:
            
            if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
                    
                my_dict = {"correct": raw_dict["sequences"]["counts"]["correct"], 
                           "total": raw_dict["sequences"]["counts"]["total"]}
                my_list.append(Row(**my_dict))
    
    return my_list

In [118]:
my_correct_total = assessments.rdd.flatMap(my_lambda_correct_total).toDF()

In [119]:
my_correct_total.registerTempTable('ct')

In [120]:
spark.sql("select * from ct limit 10").show()

+-------+-----+
|correct|total|
+-------+-----+
|      2|    4|
|      1|    4|
|      3|    4|
|      2|    4|
|      3|    4|
|      5|    5|
|      1|    1|
|      5|    5|
|      4|    4|
|      0|    5|
+-------+-----+



In [121]:
spark.sql("select correct / total as score from ct limit 10").show()

+-----+
|score|
+-----+
|  0.5|
| 0.25|
| 0.75|
|  0.5|
| 0.75|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  0.0|
+-----+



In [122]:
spark.sql("select avg(correct / total)*100 as avg_score from ct limit 10").show()

+-----------------+
|        avg_score|
+-----------------+
|62.65699745547047|
+-----------------+



In [123]:
spark.sql("select stddev(correct / total) as standard_deviation from ct limit 10").show()

+-------------------+
| standard_deviation|
+-------------------+
|0.31086692286170553|
+-------------------+



# My code starts here:



## Included Files

### 1. docker-compose.yml

In [124]:
dc = open('docker-compose.yml')
print(dc.read())

---
version: '2'
services:
  zookeeper:
    image: confluentinc/cp-zookeeper:latest
    environment:
      ZOOKEEPER_CLIENT_PORT: 32181
      ZOOKEEPER_TICK_TIME: 2000
    expose:
      - "2181"
      - "2888"
      - "32181"
      - "3888"

  kafka:
    image: confluentinc/cp-kafka:latest
    depends_on:
      - zookeeper
    environment:
      KAFKA_BROKER_ID: 1
      KAFKA_ZOOKEEPER_CONNECT: zookeeper:32181
      KAFKA_ADVERTISED_LISTENERS: PLAINTEXT://kafka:29092
      KAFKA_OFFSETS_TOPIC_REPLICATION_FACTOR: 1
    expose:
      - "9092"
      - "29092"

  cloudera:
    image: midsw205/cdh-minimal:latest
    expose:
      - "8020" # nn
      - "50070" # nn http
      - "8888" # hue
    #ports:
    #- "8888:8888"

  spark:
    image: midsw205/spark-python:0.0.5
    stdin_open: true
    tty: true
    volumes:
      - ~/w205:/w205
    expose:
      - "8888"
    ports:
      - "8888:8888"
    command: bash
    depends_on:
      - cloudera
    environment:
      HADOOP_NAMENODE: cloudera

### 2. history.txt

In [140]:
history = open('jupyter-history.txt')
print(history.read())

    1  ls -la
    2  ls
    3  docker pull midsw205/base
    4  mkdir ~/w205
    5  docker run -it --rm -v ~/w205:/w205 midsw205/base:latest bash
    6  cd ~/w205
    7  ls -l
    8  git clone https://github.com/mids-w205-crook/course-content.git
    9  ls -lk
   10  ls -l
   11  cd course-content/
   12  ls
   13  ls -l
   14  02-Working-with-Data/
   15  cd 02-Working-with-Data/
   16  ls
   17  open async-videos.md 
   18  xdg-open async-videos.md 
   19  sudo apt xdg
   20  sudo apt xdg-open
   21  cd ~
   22  cd /w205
   23  ls
   24  cd w205
   25  pwd
   26  ls -l
   27  git clone https://github.com/mids-w205-crook/signup-jacob-sycoff.git
   28  ls -l
   29  cd
   30  cd w205
   31  ls
   32  cd signup-jacob-sycoff/
   33  git status
   34  git branch assignment
   35  git status
   36  git checkout assignment
   37  git status
   38  vi README.md 
   39  VI README.MD
   40  vi README.md 
   41  nano README.md 
   42  vi README.md 
   43  git status
   44  git add README.md 
   

## The set up process:

#### 1) command to bring up the cluster
```bash
docker-compose up -d
```
###### a) check open docker-compose containers
```bash
    docker-compose ps
```
###### b) check open docker containers
```bash
    docker ps -a
```

#### 2) command to create the kafka topic with a meaningful name (most students choose assessments)
```bash
docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

docker-compose exec kafka kafka-topics --describe --topic assessments --zookeeper zookeeper:32181
```

#### 3) command to publish the assessments json data to the kafka topic using kafkacat
```bash
docker-compose exec mids bash -c "cat /w205/project-2-jacob-sycoff/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments && echo 'Published assessments json data to kafka topic using kafkacat.'"

docker-compose exec mids bash -c "kafkacat -C -b kafka:29092 -t assessments -o beginning -e"
```

#### 4) command to shutdown the cluster

```spark
exit
```

### Bringing up jupyter

#### 1) Enter spark bash:
```bash
docker-compose exec spark bash
```
#####  a) In spark bash, link w205 with w205:
```spark
    ln -s /w205 w205
```
##### b) Exit spark bash:
```spark
exit
```

#### 2) Create Jupyter environment:
```bash
docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark
```

#### 3) Change 0.0.0.0 in resulting link to external IP address in google cloud, then paste into an incognito Google Chrome window to minimize chances of errors. Jupyter notebook will come up and pyspark will work within it.

## Analysis Begins Here:

#### 1) How many assessments are in the dataset?

In [154]:
num_assessments = spark.sql("select count(*) from (select sequences_id from sequences)").show()

+--------+
|count(1)|
+--------+
|    3280|
+--------+



There are 3280 assessments in the dataset

#### 2) What's the name of your Kafka topic? How did you come up with that name?

The name of my Kafka topic is assessments because of the contents of the information within the dataset - infomration from a service that delivers assessments

#### 3) How many people took Learning Git?

In [127]:
spark.sql("select count from (select exam_name, count(*) as count from(select exam_name from assessments) group by exam_name) where exam_name='Learning Git' ").show(truncate = False)

+-----+
|count|
+-----+
|394  |
+-----+



394 people took "Learning Git"

#### 4) What is the least common course taken?

In [128]:
spark.sql("select exam_name from(select exam_name, count(*) as count from(select exam_name from assessments) group by exam_name order by count asc) limit 4 ").show(truncate = False)

+-------------------------------------------------+
|exam_name                                        |
+-------------------------------------------------+
|Native Web Apps for Android                      |
|Learning to Visualize Data with D3.js            |
|Nulls, Three-valued Logic and Missing Information|
|Operating Red Hat Enterprise Linux Servers       |
+-------------------------------------------------+



The four above courses all are the least common with only 1 participant

#### 5? What is the most common course taken?

In [129]:
spark.sql("select exam_name from(select exam_name, count(*) as count from(select exam_name from assessments) group by exam_name order by count desc) limit 1").show(truncate = False)

+------------+
|exam_name   |
+------------+
|Learning Git|
+------------+



## Additional Questions:

#### 1) Has the number of tests taken increased over time?

In [130]:
spark.sql("select year_month, count(*) from(select started_at, substring(started_at, 0, 7)as year_month from assessments order by year_month asc) group by year_month").show(truncate = False)

+----------+--------+
|year_month|count(1)|
+----------+--------+
|2017-11   |521     |
|2017-12   |1516    |
|2018-01   |1243    |
+----------+--------+



The tests taken over time increased and then decreased, but the information is not useful because there is only data present from 3 months. It is therefore my reccomendation to collect and store data consistently for every test given every month

#### 2) Are there more certification or non-certification exams?

In [131]:
spark.sql("select certification, count(*) as count from(select certification from assessments) group by certification").show(truncate = False)

+-------------+-----+
|certification|count|
+-------------+-----+
|null         |132  |
|false        |3148 |
+-------------+-----+



The only value given for certification is false, while a small subset have a null value for certification.  This bring into question the reason for the collection of this data. Should the company remove this field? Does the company have a potential opportunity to bringing in business for certification exams?

#### 3) What percent of tests were left at least 25% unanswered or incomplete?

In [132]:
#I create the function that will allow me to create a table with columns showing unanswered and total questions per test, apply it, and analyze the resulting table

In [133]:
def my_lambda_unanswered_total(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:
        
        if "counts" in raw_dict["sequences"]:
            
            if "unanswered" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"] and "incomplete" in raw_dict["sequences"]["counts"]:
                                    
                my_dict = {"unanswered": raw_dict["sequences"]["counts"]["unanswered"], 
                           "incomplete": raw_dict["sequences"]["counts"]["incomplete"],
                           "total": raw_dict["sequences"]["counts"]["total"]}
                my_list.append(Row(**my_dict))
    
    return my_list

In [147]:
my_unanswered_total = assessments.rdd.flatMap(my_lambda_unanswered_total).toDF()

In [148]:
my_unanswered_total.registerTempTable('ut')

In [149]:
spark.sql("select count(*)/(select count(*) from ut) as pct_25_or_more_pct_not_done from (select pct_not_done from (select *, unanswered_incomplete/total as pct_not_done from(select unanswered, incomplete,unanswered + incomplete as unanswered_incomplete, total from ut)) where pct_not_done >=.25)").show()

+---------------------------+
|pct_25_or_more_pct_not_done|
+---------------------------+
|        0.27541984732824426|
+---------------------------+



In [150]:
my_unanswered_total.write.mode('overwrite').parquet("/tmp/my_unanswered_total")

In [151]:
ut = spark.read.parquet("/tmp/my_unanswered_total")

In [152]:
ut.createOrReplaceTempView("ut")

In [153]:
ut = spark.sql("select count(*)/(select count(*) from ut) as pct_25_or_more_pct_not_done from (select pct_not_done from (select *, unanswered_incomplete/total as pct_not_done from(select unanswered, incomplete,unanswered + incomplete as unanswered_incomplete, total from ut)) where pct_not_done >=.25)").show()

+---------------------------+
|pct_25_or_more_pct_not_done|
+---------------------------+
|        0.27541984732824426|
+---------------------------+



About 28% of tests have at least 25% of their questions left incomplete or unanswered